In [1]:
import random

from data_generator.csv_data_processor import CSVDataProcessor
from utils.util_funcs import get_row_count, display_df, remove_data, verify_ranking, verify_ranking_counts
from utils.read_write import read_postgres_table
from spark_instance import spark
from pyspark.sql import Window

from pyspark.sql.functions import col, lower, lit, rand, array, floor, concat, when, row_number, concat_ws, date_format, split, size
  

/Users/jacobbickerstaff/Desktop/Development/health-data-etl-kaggle/venv/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
24/03/12 21:27:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# csv_reader = CSVDataProcessor(spark, "data/healthcare_dataset.csv")
# 
# # Read the CSV file
# raw_df = csv_reader.run()


In [3]:
# min_age_days = 1 * 365  # Minimum age in days (18 years)
# max_age_days = 90 * 365  # Maximum age in days (90 years)
# 
# raw_df = (raw_df.withColumn("DOB", expr(f"date_sub(current_date(), CAST(round(rand() * ({max_age_days} - {min_age_days}) + {min_age_days}) AS INT))"))
#         .withColumn("Age", floor(datediff(current_date(), col("DOB")) / 365.25)))




In [4]:
# above is commented out as it's saved in the database:
df = read_postgres_table("dob_age_raw_data")

In [5]:
# df.where(col("Age") < 16).show()

In [6]:
from constants.admission_types_dataset import admission_mapping, AdmissionTypes
flattened = [
    (
        top_level.name,
        sub_level_key.name,
        stay_type,
        sub_level_info.get("conditions",),  
        sub_level_info.get("tests"),  
        random.choice(sub_level_info.get("doctors"))
    )
    for top_level, sub_level_dict in admission_mapping.items()
    for sub_level_key, sub_level_info in sub_level_dict.items()
    for stay_type in (sub_level_info["stay_types"] if isinstance(sub_level_info, dict) else sub_level_info)
]



In [7]:
from constants.condition_probabilities import condition_age_probability_dict

flattened_condition_probabilities = [
    (
        sub_admission, 
        condition, 
        float(age_range[0]), 
        float(age_range[1]) if len(age_range) > 1 else float('inf'), 
        float(probability)  
    )
    for sub_admission, conditions in condition_age_probability_dict.items()
    for condition, age_prob_list in conditions.items()
    for age_range, probability in age_prob_list
]



In [8]:
columns = ["sub_level_admission", "condition", "age_min", "age_max", "probability"]
condition_prob_df = spark.createDataFrame(data=flattened_condition_probabilities, schema=columns)


In [11]:
condition_prob_df.show()

+-------------------+--------------------+-------+-------+-----------+
|sub_level_admission|           condition|age_min|age_max|probability|
+-------------------+--------------------+-------+-------+-----------+
|             CANCER|      Bladder Cancer|    0.0|   10.0|        0.0|
|             CANCER|      Bladder Cancer|   11.0|   17.0|     1.8E-4|
|             CANCER|      Bladder Cancer|   18.0|   25.0|     2.0E-4|
|             CANCER|      Bladder Cancer|   25.0|   34.0|    0.00224|
|             CANCER|      Bladder Cancer|   35.0|   44.0|    0.00448|
|             CANCER|      Bladder Cancer|   45.0|   59.0|    0.00673|
|             CANCER|      Bladder Cancer|   45.0|   54.0|    0.00897|
|             CANCER|      Bladder Cancer|   55.0|   64.0|    0.02018|
|             CANCER|      Bladder Cancer|   65.0|   75.0|    0.02242|
|             CANCER|      Bladder Cancer|   76.0|   80.0|     0.0426|
|             CANCER|Brain and Nervous...|    0.0|   10.0|        0.0|
|     

In [12]:
df.show()

+--------------------+---+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+----------+
|                name|Age|gender|blood_type|medical_condition|date_of_admission|            doctor|            hospital|insurance_provider|    billing_amount|room_number|admission_type|discharge_date| medication|test_results|       DOB|
+--------------------+---+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+----------+
|     Tiffany Ramirez| 39|Female|        O-|         Diabetes|       2022-11-17|    Patrick Parker|    Wallace-Hamilton|          Medicare| 37490.98336352819|        146|      Elective|    2022-12-01|    Aspirin|Inconclusive|1984-09-30|
|         Ruben Burns| 16|  Male|        O+|        

In [13]:
"""
STEP 1 
    From the flattened data list we wish to create a DataFrame. This contains all the possible combinations for the given 
    top level admissions, sub level admissions, stay types and list of tests available from the admission_mapping, stay_type and admission_tests lists or dictionaries in admission_types_test_dataset.py
"""
mapping_df = spark.createDataFrame(flattened, ["top_level_admission", "sub_level_admission", "stay_type", "conditions", "possible_tests", "doctor"])

display_df(mapping_df, 45)

,top_level_admission,sub_level_admission,stay_type,conditions,possible_tests,doctor
0,EMERGENCY,INJURY_RTC,Inpatient,"[Whiplash, Concussions, Contusions (bruises), ...","[X-rays, CT scans, MRI, Ultrasound, Blood tests]",Dr. James Brown
1,EMERGENCY,INJURY_RTC,Day Patient,"[Whiplash, Concussions, Contusions (bruises), ...","[X-rays, CT scans, MRI, Ultrasound, Blood tests]",Dr. Candace Brown
2,EMERGENCY,SELF_INFLICTED,Inpatient,"[Lacerations or cuts, Overdose, Burning, Self-...","[Psychological assessment, X-rays (for physica...",Dr. Kara Mcmillan
3,EMERGENCY,CARDIOLOGY,Inpatient,"[Hypertension (High Blood Pressure), Coronary ...","[ECG, Echocardiogram, Stress tests, Cardiac ca...",Dr. Megan Wood
4,EMERGENCY,CARDIOLOGY,Day Patient,"[Hypertension (High Blood Pressure), Coronary ...","[ECG, Echocardiogram, Stress tests, Cardiac ca...",Dr. Amy Hicks
5,EMERGENCY,NEUROLOGY,Inpatient,"[Migraines, Epilepsy, Parkinson's Disease, Alz...","[MRI or CT scans of the brain, Electroencephal...",Dr. Karen Ford
6,EMERGENCY,NEUROLOGY,Day Patient,"[Migraines, Epilepsy, Parkinson's Disease, Alz...","[MRI or CT scans of the brain, Electroencephal...",Dr. Fred Briggs
7,EMERGENCY,NEUROLOGY,Outpatient,"[Migraines, Epilepsy, Parkinson's Disease, Alz...","[MRI or CT scans of the brain, Electroencephal...",Dr. Nicole Hall
8,EMERGENCY,GASTROENTEROLOGY,Inpatient,"[Gastro-oesophageal Reflux Disease (GORD), Irr...","[Endoscopy, Colonoscopy, Blood tests, Stool te...",Dr. Kim Burch
9,EMERGENCY,GASTROENTEROLOGY,Day Patient,"[Gastro-oesophageal Reflux Disease (GORD), Irr...","[Endoscopy, Colonoscopy, Blood tests, Stool te...",Dr. Claudia Koch


In [14]:
joined_tbl = condition_prob_df.join(mapping_df, on="sub_level_admission", how="left")

In [15]:
joined_tbl.show(2000)

+-------------------+--------------------+-------+--------+-----------+-------------------+-----------+--------------------+--------------------+--------------------+
|sub_level_admission|           condition|age_min| age_max|probability|top_level_admission|  stay_type|          conditions|      possible_tests|              doctor|
+-------------------+--------------------+-------+--------+-----------+-------------------+-----------+--------------------+--------------------+--------------------+
|         CARDIOLOGY|           Aneurysms|    0.0|    10.0|        0.0|  HOSPITAL_REFERRAL|Day Patient|[Hypertension (Hi...|[ECG, Echocardiog...|   Dr. Brian Beltran|
|         CARDIOLOGY|           Aneurysms|    0.0|    10.0|        0.0|  HOSPITAL_REFERRAL|  Inpatient|[Hypertension (Hi...|[ECG, Echocardiog...|    Dr. Jonathan May|
|         CARDIOLOGY|           Aneurysms|    0.0|    10.0|        0.0|          EMERGENCY|Day Patient|[Hypertension (Hi...|[ECG, Echocardiog...|       Dr. Amy Hicks

In [11]:
"""
STEP 2 
    Create a list of admission_types randomly assign this to the original patient in the original data set, 
    whilst dropping the original admission_type column. Then join with mapping_df on top_level_admission col to give access to possible 
    conditions, mappings and so on.
"""
admission_type_names = [member.name for member in AdmissionTypes]

print(admission_type_names)

keys_array = array([lit(name) for name in admission_type_names])

# Define constants and conditions
female_only = ["MATERNITY", 'obstetrics']
is_female = lower(col('gender')) == 'female'
is_pediatric = col("Age") < 18
is_geriatric = (col("Age") >= 65) & (col("sub_level_admission") == "GERIATRICS")

df = (df.withColumn("top_level_admission", keys_array[floor(rand() * len(admission_type_names))]).drop("admission_type")
        .withColumn("is_female", is_female)
        .withColumn("is_pediatric", is_pediatric)
        .drop("doctor", "medical_condition", "test_results", "medication")
      )
get_row_count(df, True)
df = df.join(mapping_df, "top_level_admission", "inner")
get_row_count(df, True)
display_df(df, 999)

['ELECTIVE', 'EMERGENCY', 'GP_REFERRAL', 'HOSPITAL_REFERRAL', 'SELF_REFERRAL']
10000
155368


,top_level_admission,name,Age,gender,blood_type,date_of_admission,hospital,insurance_provider,billing_amount,room_number,discharge_date,DOB,is_female,is_pediatric,sub_level_admission,stay_type,conditions,possible_tests,doctor
0,ELECTIVE,Ruben Burns,16,Male,O+,2023-06-01,"Burke, Griffin and Cooper",UnitedHealthcare,47304.064845,404,2023-06-15,2007-04-05,False,True,MATERNITY,Inpatient,"[Labour and Delivery, Scheduled Caesarean Sect...","[Ultrasound, Blood tests, Glucose tolerance te...",Dr. Ashley Jones
1,ELECTIVE,Ruben Burns,16,Male,O+,2023-06-01,"Burke, Griffin and Cooper",UnitedHealthcare,47304.064845,404,2023-06-15,2007-04-05,False,True,MATERNITY,Day Patient,"[Labour and Delivery, Scheduled Caesarean Sect...","[Ultrasound, Blood tests, Glucose tolerance te...",Dr. Kenneth Smith
2,ELECTIVE,Ruben Burns,16,Male,O+,2023-06-01,"Burke, Griffin and Cooper",UnitedHealthcare,47304.064845,404,2023-06-15,2007-04-05,False,True,MATERNITY,Outpatient,"[Labour and Delivery, Scheduled Caesarean Sect...","[Ultrasound, Blood tests, Glucose tolerance te...",Dr. Carol Marsh
3,ELECTIVE,Patty Norman,22,Female,O-,2019-05-16,Brown Inc,Blue Cross,13546.817249,384,2019-06-02,2001-08-18,True,False,MATERNITY,Inpatient,"[Labour and Delivery, Scheduled Caesarean Sect...","[Ultrasound, Blood tests, Glucose tolerance te...",Dr. Ashley Jones
4,ELECTIVE,Patty Norman,22,Female,O-,2019-05-16,Brown Inc,Blue Cross,13546.817249,384,2019-06-02,2001-08-18,True,False,MATERNITY,Day Patient,"[Labour and Delivery, Scheduled Caesarean Sect...","[Ultrasound, Blood tests, Glucose tolerance te...",Dr. Kenneth Smith
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,ELECTIVE,Jake Matthews,88,Female,B-,2018-12-19,Gilbert-Sanchez,UnitedHealthcare,15662.350218,486,2019-01-06,1935-03-04,True,False,MATERNITY,Day Patient,"[Labour and Delivery, Scheduled Caesarean Sect...","[Ultrasound, Blood tests, Glucose tolerance te...",Dr. Kenneth Smith
995,ELECTIVE,Jake Matthews,88,Female,B-,2018-12-19,Gilbert-Sanchez,UnitedHealthcare,15662.350218,486,2019-01-06,1935-03-04,True,False,MATERNITY,Outpatient,"[Labour and Delivery, Scheduled Caesarean Sect...","[Ultrasound, Blood tests, Glucose tolerance te...",Dr. Carol Marsh
996,ELECTIVE,Kathryn Mcpherson PhD,61,Male,AB-,2023-03-10,Brown LLC,UnitedHealthcare,24735.619413,222,2023-03-29,1962-02-11,False,False,MATERNITY,Inpatient,"[Labour and Delivery, Scheduled Caesarean Sect...","[Ultrasound, Blood tests, Glucose tolerance te...",Dr. Ashley Jones
997,ELECTIVE,Kathryn Mcpherson PhD,61,Male,AB-,2023-03-10,Brown LLC,UnitedHealthcare,24735.619413,222,2023-03-29,1962-02-11,False,False,MATERNITY,Day Patient,"[Labour and Delivery, Scheduled Caesarean Sect...","[Ultrasound, Blood tests, Glucose tolerance te...",Dr. Kenneth Smith


In [ ]:
"""
STEP 3 
    Create a rowq ranking using a unique cols, created from stay_name and unique_id. 
    Verification done below. 
    
"""

df =  df.withColumn('stay_name', 
                   when(col('stay_type') == 'out_patient', concat(col('name'), lit('_out_patient')))
                   .when(col('stay_type') == 'inpatient', concat(col('name'), lit('_inpatient')))
                   .when(col('stay_type') == 'day_patient', concat(col('name'), lit('_day_patient')))
                   .otherwise(col('name'))
                  ).withColumn("unique_id", concat_ws("_", "name", date_format("DOB", "yyyyMMdd")))


# Define a window specification that partitions data by 'top_level_admission' (or another unique patient identifier if needed)
windowSpec = Window.partitionBy('stay_name', 'unique_id').orderBy(rand())

# Assign row numbers within each partition in a random order
ranked_df = df.withColumn("row_num", row_number().over(windowSpec))


In [ ]:
ranked_df.createGlobalTempView("ranked_df")

unique_dobs_df = spark.sql("""
WITH NameCounts AS (
    SELECT name
    FROM global_temp.ranked_df
)

SELECT DISTINCT r.name, r.unique_id
FROM global_temp.ranked_df r
JOIN NameCounts n ON r.name = n.name
ORDER BY r.name
LIMIT 10
""")

In [ ]:
# COSTLY WAY TO VERIFY THAT THE RANK WORKS BELOW....  due to .collect() and .count() in verify_ranking_counts() function

# Example usage:
unique_names = [row['name'] for row in unique_dobs_df.select("name").collect()]
unique_ids = [row['unique_id'] for row in unique_dobs_df.select("unique_id").collect()]
verify_ranking_counts(df, ranked_df, unique_names, unique_ids)

# this function is faster than above
verify_ranking(df, ranked_df)


In [ ]:
"""
Step 4: 
    Sort out geriatrics data and verify that individuals are not geriatrics 
"""
ranked_df = ranked_df.withColumn("is_geriatric", is_geriatric)

not_geriatric_df = ranked_df.where((col('sub_level_admission') == "GERIATRICS") &( col("is_geriatric") == False))

filtered_df = remove_data(ranked_df, not_geriatric_df, (col('sub_level_admission') == "GERIATRICS"), ( col("is_geriatric") == False))


In [ ]:
"""
Step 5: 
    Sort out geriatrics data and verify that individuals who are not female and should not have a female sub level admission 
"""

not_female_df = filtered_df.where((col("sub_level_admission").isin(female_only)) & (col("is_female") == False))
filtered_df_female = remove_data(filtered_df, not_female_df, (col("gender") == "Male"),  (col("sub_level_admission").isin(female_only)))


In [ ]:
# filter pediatric patients who cannot be pregnant (based on legal age in the UK, 16) No assumption made an individual cannot choose to get pregnant before this age. 
# upper age bound defined  here: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4822427/#:~:text=Menopause%20typically%20occurs%20between%2045,reproducing%20many%20years%20before%20menopause.
# between 45 and 55 upper bounding will be 50. # Possiblilty to include outliers in a new func in the future.
under16_female_df = filtered_df_female.where((col("Age") < 16) &
                                             (col("Age") <= 50) &
                                             (col("sub_level_admission") == "MATERNITY")
                                             ).orderBy("Age")

In [ ]:
df = remove_data(filtered_df_female, under16_female_df, (col("sub_level_admission") == "MATERNITY"),
                 (col("Age") < 16) & (col("Age") > 50))



In [ ]:

split_col = split(df['name'], ' ')

df_renamed = (df.withColumn('name', 
                    when(size(split_col) == 4, concat(split_col.getItem(1).substr(1, 1), lit('. '), split_col.getItem(2).substr(1, 1), lit('. '), split_col.getItem(3)))
                    .when(size(split_col) == 3, concat(split_col.getItem(1).substr(1, 1), lit('. '), split_col.getItem(2)))
                    .otherwise(
                        concat(split_col.getItem(0).substr(1, 1), lit('. '), split_col.getItem(1))
                               )
                    )
      )


In [ ]:
df_renamed.show()

In [ ]:
(df_renamed.select("name", "DOB", "Age", "gender", "blood_type", "date_of_admission", "discharge_date", "top_level_admission", "sub_level_admission", "possible_tests", "conditions", "doctor", "hospital", "room_number", "insurance_provider", "billing_amount", "stay_type", "is_female", "is_geriatric", "is_pediatric", "stay_name", "row_num", "unique_id")
 # .where(col("stay_name") =="Abigail Lamb")
 .show(n=200))

In [ ]:

from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

sub_selection_cols = ["Age", "sub_level_admission", "conditions", "row_num", "unique_id"]

# df_subset = df_renamed.select(*sub_selection_cols)
# 
# choose_condition_udf = udf(choose_condition, StringType())
# df_test = df_subset.withColumn("assigned_condition", choose_condition_udf(df_subset["age"], df_subset["sub_level_admission"], df_subset["conditions"]))


In [ ]:
# df_subset.persist()
# df_subset.show()  # The action that triggers the computation


In [ ]:
# df_test.persist()
# df_test.show(n=20)

In [ ]:
df_new = df_renamed.drop("row_num")

windowSpec = Window.partitionBy('stay_name', 'unique_id').orderBy(rand())

# Assign row numbers within each partition in a random order
df_new_part = df_new.withColumn("row_num", row_number().over(windowSpec))

df_new_part.select(["name", "DOB", "Age", "gender", "blood_type", "date_of_admission", "discharge_date", "top_level_admission", "sub_level_admission", "possible_tests", "conditions", "doctor", "hospital", "room_number", "insurance_provider", "billing_amount", "stay_type", "is_female", "is_geriatric", "is_pediatric", "stay_name", "row_num", "unique_id"]).where(col("stay_name") =="Tiffany Ramirez").show() 

In [ ]:
get_row_count(df_new_part)

In [ ]:
["name", "DOB", "Age", "gender", "blood_type", "date_of_admission", "discharge_date", "top_level_admission", "sub_level_admission", "possible_tests", "conditions", "doctor", "hospital", "room_number", "insurance_provider", "billing_amount", "stay_type", "is_female", "is_geriatric" "is_pediatric", "stay_name", "row_num", "unique_id"]

In [ ]:
# join_with_condition_prob_df = df_new_part.join(condition_prob_df, 
#                                                on=["sub_level_admission"], 
#                                                how="inner")
# join_with_condition_prob_df = join_with_condition_prob_df.filter(
#     (col("Age") >= col("age_min")) &
#     (col("Age") <= col("age_max"))
# )

In [ ]:
df_new_part.show(n=2000)

In [ ]:
# TODO:  
# medical condition to be  chosen 
# tests to be chosen
# admission date to be checked again dob,  
# TODO filter on is pediatric, geriatric and is_female to be done here and same people with dob? needs  to be considered 
# drop stay_name and unique_id 


In [ ]:
# Filter to keep only the top-ranked row within each partition
ranked_df = filtered_df_female

In [ ]:
# from pyspark.sql.types import StringType
# get_row_count(df, True)
df = join_with_condition_prob_df.select([col(c).cast(StringType()).alias(c) for c in join_with_condition_prob_df.columns])
df.show()
df.repartition(10).write.csv('./temp_data/join_with_condition_prob_df/renamed.csv', mode = 'overwrite', header=True)

In [ ]:
# TODO initialise first name to make data more realistic for name columns i.e. Daniel Mccoy is seen a Male name but here its Female

In [ ]:
spark.stop()

In [ ]:
dict_ = admission_mapping.get("emergency")